In [205]:
import os , re, os, datetime, dateutil, pytz, copy
from dateutil import parser
import ACILib

logfile = '/home/jovyan/HTTSDashboard/logs/RabbitMQ_ACI_Event.log'
logfiletime = os.path.getmtime(logfile)
logfiledatetime = datetime.datetime.fromtimestamp(logfiletime)
logfiledatetime

datetime.datetime(2020, 6, 9, 5, 46, 2, 490929)

In [239]:
def GetShiftHour(date='2020-04-05',shift='apjc'):
    
    date_datetime = parser.parse(date)

    ShiftTime = {
        '2020Summer':['2019-10-10','2020-03-28'],
        '2020InterToWinter':['2020-03-29','2020-04-04'],
        '2020Winter':['2020-04-05','2020-09-01'],
        '2020InterToSummer':['2020-09-02','2020-10-01']
    }
    ShiftHour = {
        'Summer':[1,7],
        'InterToWinter':[0,6],
        'Winter':[0,6],
        'InterToSummer':[1,7]
    }
    
    AESTorAEDT = {
        '10':[
            ['2020-04-04','2020-10-04'],
            ['2021-04-05','2021-10-03']
        ],
        '11':[
            ['2019-10-27','2020-04-04'],
            ['2020-10-05','2021-04-04']
        ]
    }
    
    GMT_start = -1
    ShiftStartHour = []
    
    for key,period in ShiftTime.items():
        year,sfhit,start_datetime,end_datetime = key[:4],key[4:],parser.parse(period[0]),parser.parse(period[1])
        #print("GetShiftHour - {} v.s. {} {} {} {}".format(date, year,sfhit,period[0],period[1]))
        if date_datetime >= start_datetime and date_datetime <= end_datetime:
            ShiftStartHour = ShiftHour[key[4:]]  #[1, 7]
            break
            
    for gmthour,periodlist in AESTorAEDT.items():
        for period in periodlist:
            #print(period)
            if date_datetime >= parser.parse(period[0]) and date_datetime <= parser.parse(period[1]):
                GMT_start = gmthour
                break
    
    if shift == 'emea':
        ShiftStartHour[0] = ShiftStartHour[0]+6
        ShiftStartHour[1] = ShiftStartHour[1]+6

    #print("ShiftStartHourInGMT {} StartGMT {}".format(ShiftStartHour[0],GMT))
    return ShiftStartHour,GMT_start

GetShiftHour(date='2020-06-09',shift='apjc')

([0, 6], '10')

In [277]:
def isOnShift(timestamp,timezone='America/Los_Angeles',shift='apjc',before_shift_hours_offset = 0, after_shift_hours_offset = 0, debug=False):
    
    '''
    before_shift_hours_offset is the hours before shift if would like to be considered to be onshift, default is 0
    after_shift_hours_offset is the hours after shift if would like to be considered to be onshift, default is 0
    before_shift_hours_offset and after_shift_hours_offset is majorly used for dispatched case that dispatched one hour prior to shift would be considered.
    
    local = pytz.timezone ("America/Los_Angeles")
    naive = datetime.datetime.strptime ("2001-2-3 10:11:12", "%Y-%m-%d %H:%M:%S")
    local_dt = local.localize(naive, is_dst=None)
    utc_dt = local_dt.astimezone(pytz.utc)
    India Standard Time (Asia/Kolkata)
    '''
    
    date, time = timestamp.split()
    timezone = timezone.strip()
    
    local = pytz.timezone (timezone)
    local_dt = local.localize(parser.parse(timestamp))
    local_gmt_offset = local_dt.utcoffset().total_seconds()/60/60
    utc_dt = local_dt.astimezone(pytz.utc)
    #print("InputTime:{} TimeZone:{} GMTOffset:{} UTCTime:{}".format(
    #    timestamp,timezone,local_gmt_offset,utc_dt.strftime("%Y-%m-%d %H:%M:%S")))
    utc_date = utc_dt.strftime("%Y-%m-%d")
    utc_time = utc_dt.strftime("%H:%M:%S")
    hour = utc_dt.hour
    minute = utc_dt.minute
    
    shift_hour, GMT_start = GetShiftHour(date,shift)
    GMT_start = int(GMT_start)
    orig_shift_hour = shift_hour.copy()
    ###### If offset is considered, plus 24 as event might be at 23:00 - 24:00 the day before ######
    if before_shift_hours_offset != 0 or after_shift_hours_offset != 0:
        shift_hour[0] = shift_hour[0] + 24 - before_shift_hours_offset 
        shift_hour[1] = shift_hour[1] + after_shift_hours_offset + 24
        if hour <= 7:
            hour = hour + 24
            
    if debug:
        print("isOnShift:Shift Hour {} {} Start GMT {}".format(shift_hour,orig_shift_hour,GMT_start))
        print("isOnShift:Hour {} Minute {}".format(hour,minute))
        
    if hour >= shift_hour[0] and hour < shift_hour[1]:
        if debug:
            print("isOnShift:IsOnShift {}".format(utc_date+' '+utc_time))
        return "On", (utc_dt + datetime.timedelta(hours=GMT_start+orig_shift_hour[0])).strftime("%Y-%m-%d %H:%M:%S")
    else:
        if debug:
            print("isOnShift:OffShift {}".format(utc_date+' '+utc_time))
        return "Off", (utc_dt + datetime.timedelta(hours=(GMT_start+orig_shift_hour[0]))).strftime("%Y-%m-%d %H:%M:%S")
    
isOnShift(timestamp = '2020-06-08 09:30:00',timezone = 'Europe/Istanbul',shift='emea',before_shift_hours_offset=0,debug = True)

isOnShift:Shift Hour [6, 12] [6, 12] Start GMT 10
isOnShift:Hour 6 Minute 30
isOnShift:IsOnShift 2020-06-08 06:30:00


('On', '2020-06-08 22:30:00')

In [279]:
#It was exected in the jupyter notebook container , path is different from flask container.

date = '2020-06-08'
#date = datetime.datetime.today().strftime("%Y-%m-%d")
#current_hour = datetime.datetime.now().hour
#if current_hour >= 6:
#    exit()

jupyter_container_path = '/home/jovyan'
flask_container_path = ''
current_container_path = jupyter_container_path

sourcefile = current_container_path + '/HTTSDashboard/logs/RabbitMQ_ACI_Event.log'
sourcefiletime = os.path.getmtime(sourcefile)
sourcefiledatetime = datetime.datetime.fromtimestamp(sourcefiletime)
sourcefileformattime = sourcefiledatetime.strftime("%Y-%m-%d %H:%M:%S")

commentfile = current_container_path + '/HTTSDashboard/logs/ACI/data/comment.txt'
casestatsdir = current_container_path + '/HTTSDashboard/logs/ACI/data/'
engrstatsdir = current_container_path + '/HTTSDashboard/logs/ACI/data/'
dailyreportfiledir = current_container_path + '/HTTSDashboard/logs/ACI/data/report/'
dailyreportfilename = date+'.txt'
dailyreportstring = ""


SYDEngrList = ['tonzeng','minkwong','siddhp','wilchong','debabbar','lindawa','junwa','annelso2','ikarvoun']
BGLEngrList = ['deepaky','knagavol','jawalia','prpratee','maveer','raghb','roagraw2','shparanj','vkalmath']
BGLOtherEngrList = ['anirukas','reperuma','deepakba','bharatkc','kdoodi']
SYDOtherEngrList = ['zdazhi','zmeng']
CSSEngrList = []
OnShiftEngrList = []

ACI_InterestQueueName = ['WW-ACI-Solutions','CX-APJC-BLR-ACI-SSPT','CX-APJC-SYD-ACI-SSPT','WW-Rakuten-ACI']
ACI_EU_InterestQueueName = ['EME-ACI-Solutions']

SV_InterestQueueName = ['WW-SV','WW-STORAGE']

InterestQueueName = ACI_EU_InterestQueueName

CaseQty = {'CX-APJC-BLR-ACI-SSPT':0,
           'WW-ACI-Solutions':0,
           'CX-APJC-SYD-ACI-SSPT':0,
           'WW-Rakuten-ACI':0,
           'FTS':0,'UC':0}
CaseTakenDic = {}
CaseTakenDicInclNo = {'SYD':{},'SYD_Other':{},'BGL':{},'BGL_Other':{}}
CaseTakenByEngrDic = {}
CaseStatsDic = {"SydQueueVol":0,"BGLQueueVol":0,
                "SydTakenVol":0,"BGLTakenVol":0,
                'FTSCaseVol':0,'UCCaseVol':0,
                "SydQueueVolByPri":[0,0,0,0,0,0],"BGLQueueVolByPri":[0,0,0,0,0,0], #0 FTS #1,2,3,4 Priority #5 Urgent Collab
                "SydTakenVolByPri":[0,0,0,0,0,0],"BGLTakenVolByPri":[0,0,0,0,0,0], #0 FTS #1,2,3,4 Priority #5 Urgent Collab
                "SydOnShiftEngr":1,"BGLOnShiftEngr":1,
                "SydOnShiftEngrTakenVol":0,"BGLOnShiftEngrTakenVol":1,
                "SYDHelper":0,"BGLHelper":0,
                }

with open(sourcefile) as f:
    allevents = f.read()
f.close()

shift_hour,gmt_start_hour = GetShiftHour(date='2020-06-08',shift='emea')
shift_start_hour = shift_hour[0]
shift_end_hour = shift_hour[1]
print("Shift Hour {} {} {}".format(shift_start_hour,shift_end_hour,gmt_start_hour))

allevents = allevents.split("\n")
allevents[-10:]

Shift Hour 6 12 10


['RabbitMQ_ACI_Event.log-2020-06-09 05:16:25,486-INFO-MQToRedis:689232009 3 Case_Dispatched_Out ACI // No able to take config backup To  2020-06-09 11:45:00(GMT+05:30) India Standard Time (Asia/Colombo)',
 'RabbitMQ_ACI_Event.log-2020-06-09 05:22:23,164-INFO-MQ:689204267 3 Case_Closed Error message on EPG by raghb APT-ACI-SOLUTIONS2 at 2020-06-09 05:20:13',
 'RabbitMQ_ACI_Event.log-2020-06-09 05:31:36,731-INFO-MQToRedis:689171782 3 UC_Accepted ACI 4.2(2f)  APIC Power Supply Failure Fault Code F1940 taken by krajasel UC_WORKGROUP at 2020-06-09 05:25:16',
 'RabbitMQ_ACI_Event.log-2020-06-09 05:39:29,563-INFO-MQ:689249154 3 Case_Created CX-APJC-SYD-ACI-SSPT Fault code F1527 by NULL NULL at 2020-06-09 05:38:18',
 'RabbitMQ_ACI_Event.log-2020-06-09 05:41:25,271-INFO-MQToRedis:689249154 3 Case_InQueue CX-APJC-SYD-ACI-SSPT Fault code F1527 at 2020-06-09 05:38:24',
 'RabbitMQ_ACI_Event.log-2020-06-09 05:57:34,642-INFO-MQToRedis:689249154 3 Case_Accepted Fault code F1527 taken by tonzeng APT-AC

In [280]:
def GetRawEventsFromDate(allevents,date='2020-02-27',shift_start_hour=0,shift_end_hour=6,debug=False):
    
    date = date
    shift_start_hour = shift_start_hour
    shift_end_hour = shift_end_hour
    allevents_in_date = []
    
    CaseMatchLine_Reg = re.compile(r'.*?MQToRedis:\d{9}\s\d\s.*at\s(\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2})')
    
    for line in allevents[:]:
        if date not in line:
            continue
        if re.search(CaseMatchLine_Reg,line):
            line_datetime = parser.parse(re.search(CaseMatchLine_Reg,line).group(1))
            #dispatched case might dispatch to today while not take into account
            
            if line_datetime.strftime("%Y-%m-%d") != date:
                #print("{} - {}".format(line_datetime.strftime("%Y-%m-%d"),date))
                continue
            if line_datetime.hour >= shift_start_hour and line_datetime.hour < shift_end_hour:
                allevents_in_date.append(line)
    return allevents_in_date

allevents_in_date = GetRawEventsFromDate(allevents,shift_start_hour=shift_start_hour,shift_end_hour=shift_end_hour,date='2020-06-08')
allevents_in_date[:10]

['RabbitMQ_ACI_Event.log-2020-06-08 06:03:25,908-INFO-MQToRedis:689238498 3 Case_InQueue EME-ACI-Solutions APIC controller/Leaf switch upgrade from 3.1.2m to 4.2.4i version at 2020-06-08 06:01:13',
 'RabbitMQ_ACI_Event.log-2020-06-08 06:03:37,639-INFO-MQToRedis:689238498 3 Case_InQueue EME-ACI-Solutions APIC controller/Leaf switch upgrade from 3.1.2m to 4.2.4i version at 2020-06-08 06:01:13',
 'RabbitMQ_ACI_Event.log-2020-06-08 06:04:38,441-INFO-MQToRedis:689238498 3 Case_Accepted APIC controller/Leaf switch upgrade from 3.1.2m to 4.2.4i version taken by anirukas GCE-ACI-Solutions at 2020-06-08 06:03:12',
 'RabbitMQ_ACI_Event.log-2020-06-08 06:04:46,056-INFO-MQToRedis:689238498 3 Case_Accepted APIC controller/Leaf switch upgrade from 3.1.2m to 4.2.4i version taken by anirukas GCE-ACI-Solutions at 2020-06-08 06:03:12',
 'RabbitMQ_ACI_Event.log-2020-06-08 06:04:52,128-INFO-MQToRedis:689239839 3 Case_InQueue EME-ACI-Solutions F1313 - Operational issues detected on multiple ESX hosts on co

In [287]:
def FindInQueueEvent(CaseEventsList,InterestQueueName=[],shift_start_hour=0,shift_end_hour=6,debug=False):
    
    InterestQueueName = InterestQueueName
    print("Interest Queue Name {}".format(InterestQueueName))
    #1 Case Number #2 Case Priority #3 Case Qeueue #4 Event Timestamp
    InQueue_Reg = re.compile(r'.*?MQToRedis:(\d{9})\s(\d).*Case_InQueue\s(.+?)\s.*at\s(\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2})')
    #1 Case Number #2 Case Priority #3 Event Timestamp
    FTS_Reg = re.compile(r'.*?MQToRedis:(\d{9})\s(\d).*FTS_InQueue\s.*at\s(\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2})')
    UC_Reg = re.compile(r'.*?MQToRedis:(\d{9})\s(\d).*UC_InQueue.*at\s(\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2})')
    
    CaseInQueueRawList = []
    FTSInQueueRawList = []
    UCInQueueRawList = []
    
    for line in CaseEventsList:
        if 'Case_InQueue' in line:
            #print(line)
            #break
            if re.search(InQueue_Reg,line):
                CaseInQueueRawList.append(re.search(InQueue_Reg,line).group(1)+"-~"+
                              re.search(InQueue_Reg,line).group(2)+"-~"+
                              re.search(InQueue_Reg,line).group(3)+"-~"+
                              re.search(InQueue_Reg,line).group(4))
    
    #CaseInQueueRawList = [re.search(InQueue_Reg,line).group(1)+"-~"+
    #                      re.search(InQueue_Reg,line).group(2)+"-~"+
    #                      re.search(InQueue_Reg,line).group(3)+"-~"+
    #                      re.search(InQueue_Reg,line).group(4) 
    #                      for line in CaseEventsList if 'Case_InQueue' in line]

    for line in CaseEventsList:
        if 'FTS_InQueue' in line:
            #print(line)
            if re.search(FTS_Reg,line):
                #print("Add FTS ".format(line))
                FTSInQueueRawList.append(re.search(FTS_Reg,line).group(1)+"-~"+
                              re.search(FTS_Reg,line).group(2)+"-~FTS-~"+
                              re.search(FTS_Reg,line).group(3))
                
    #FTSInQueueRawList = [re.search(FTS_Reg,line).group(1)+"-~"+
    #                     re.search(FTS_Reg,line).group(2)+"-~FTS-~"+
    #                     re.search(FTS_Reg,line).group(3) 
    #                     for line in CaseEventsList if 'FTS_InQueue' in line]
    
    UCInQueueRawList = [re.search(UC_Reg,line).group(1)+"-~"+
                         re.search(UC_Reg,line).group(2)+"-~UC-~"+
                         re.search(UC_Reg,line).group(3) 
                         for line in CaseEventsList if 'UC_InQueue' in line]
    
    ###### if the case case in before shift start and event in the shift start, it will still be recorded, need to remove ######
    #2020-03-06 01:00:19,511-INFO-MQToRedis:688593987 3 Case_InQueue WW-ACI-Solutions L2NodeAuthPolicy at 2020-03-06 00:58:47
    #Example is as above
    CaseInQueueRawRemoveBeforeShiftEventList = []
    for caseline in CaseInQueueRawList:
        #print(caseline)
        caseno, priority, queue, casetime = caseline.split("-~")
        case_datetime = parser.parse(casetime)
        if case_datetime.hour >= shift_start_hour and case_datetime.hour < shift_end_hour:
            CaseInQueueRawRemoveBeforeShiftEventList.append(caseline)
    CaseInQueueRawList = CaseInQueueRawRemoveBeforeShiftEventList    
    
    if debug:
        print("Original CaseInQueue List length {}".format(len(CaseInQueueRawList)))
        print("Original FTSInQueue List length {}".format(len(FTSInQueueRawList)))
        print("Original UCInQueue List length {}".format(len(UCInQueueRawList)))
        
    NotInterestQueueIndexList = [index for index,case in enumerate(CaseInQueueRawList) if case.split("-~")[2] not in InterestQueueName]
    
    if debug:
        print("Not Interest Queue Index list {}".format(NotInterestQueueIndexList))
    
    CaseInQueueRemoveNotInterestQueueExtList = [line for line in CaseInQueueRawList]
    
    for index in NotInterestQueueIndexList[::-1]:
        #print("Pop not interest {} from list".format(index))
        CaseInQueueRemoveNotInterestQueueExtList.pop(index)
    
    CaseInQueueRemoveNotInterestQueueExtList = list(set(CaseInQueueRemoveNotInterestQueueExtList))
    
    if debug:
        print("After removing NotInterestQueue length {}".format(len(CaseInQueueRemoveNotInterestQueueExtList)))
        #for case in CaseInQueueRemoveNotInterestQueueExtList:
        #    print(case)
    
    DuplicateInQueueIndexList = []
    for index1, raw_case_line1 in enumerate(CaseInQueueRemoveNotInterestQueueExtList):
        caseno1, priority1, queue1, time1 = raw_case_line1.split("-~")
        for index2, raw_case_line2 in enumerate(CaseInQueueRemoveNotInterestQueueExtList[index1+1:]):
            caseno2, priority2, queue2, time2 = raw_case_line2.split("-~")
            #print("Comparing {} {} {} - {} {} {}".format(caseno1,queue1,time1,caseno2,queue2,time2))
            if caseno1 == caseno2 and time1 == time2:
                #print("Found dupliate {} {}".format(index1,index1+index2+1))
                DuplicateInQueueIndexList.append(index1+index2+1)
    if debug:
        print("Duplicate Case Index list {}".format(DuplicateInQueueIndexList))
        for case in DuplicateInQueueIndexList:
            print(case)
            
    CaseInQueueRemoveDupExtList = [line for line in CaseInQueueRemoveNotInterestQueueExtList]
    for index in DuplicateInQueueIndexList[::-1]:
        #print("Pop duplicate {} from list".format(index))
        CaseInQueueRemoveDupExtList.pop(index)
    if debug:
        print("After removing DuplicateCase length {}".format(len(CaseInQueueRemoveDupExtList)))

    return CaseInQueueRemoveDupExtList+FTSInQueueRawList+UCInQueueRawList

InQueueEventList = FindInQueueEvent(allevents_in_date,InterestQueueName,shift_start_hour=shift_start_hour,shift_end_hour=shift_end_hour,debug=False)
InQueueEventList

Interest Queue Name ['EME-ACI-Solutions']


['689239839-~3-~EME-ACI-Solutions-~2020-06-08 06:02:42',
 '689240783-~3-~EME-ACI-Solutions-~2020-06-08 08:45:19',
 '689233408-~2-~EME-ACI-Solutions-~2020-06-08 11:59:13',
 '689185699-~3-~EME-ACI-Solutions-~2020-06-08 06:42:10',
 '689241320-~3-~EME-ACI-Solutions-~2020-06-08 10:05:34',
 '689233408-~2-~EME-ACI-Solutions-~2020-06-08 11:59:07',
 '689233408-~3-~EME-ACI-Solutions-~2020-06-08 06:21:37',
 '689241675-~3-~EME-ACI-Solutions-~2020-06-08 11:07:06',
 '689241375-~3-~EME-ACI-Solutions-~2020-06-08 10:15:01',
 '689239934-~3-~EME-ACI-Solutions-~2020-06-08 06:25:57',
 '689240082-~1-~EME-ACI-Solutions-~2020-06-08 07:24:56',
 '689182716-~2-~EME-ACI-Solutions-~2020-06-08 11:11:28',
 '689240082-~1-~EME-ACI-Solutions-~2020-06-08 06:55:41',
 '689241006-~3-~EME-ACI-Solutions-~2020-06-08 09:13:54',
 '689185699-~3-~EME-ACI-Solutions-~2020-06-08 06:42:03',
 '689238498-~3-~EME-ACI-Solutions-~2020-06-08 06:01:13',
 '689233408-~3-~EME-ACI-Solutions-~2020-06-08 06:21:31',
 '689154452-~1-~FTS-~2020-06-08

In [286]:
###### Find Dispatch Event ######
def FindDispatchEvent(CaseEventsList,debug=False):
    
    RawDispatchedEventList = []
    
    #Group 1.	2932-2941	689101876
    #Group 2.	2942-2943	3
    #Group 3.	2964-3033	APIC Appliance || Maintenance window / Standby (Closed Wednesday 19) 
    #Group 4.	3037-3056	2020-05-17 01:00:00
    #Group 5.	3060-3066	+05:30
    #Group 6.	3067-3102	 India Standard Time (Asia/Kolkata)
    Dispatch_Reg = re.compile(r'.*?MQToRedis:(\d{9})\s(\d).*Case_Dispatched_Out\s(.+?)To\s\s(\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2})\(GMT([+-]\d{2}:\d{2})\)(.+)')
    
    for line in CaseEventsList:
        if 'Case_Dispatched_Out' in line: 
            dispatch_line = re.search(Dispatch_Reg,line)
            if dispatch_line:
                caseno = dispatch_line.group(1)
                severity = dispatch_line.group(2)
                casetitle = dispatch_line.group(3).strip()
                timestamp = dispatch_line.group(4)
                #Two type of the timezone
                #2020-02-09 00:30:00(GMT+03:00) Europe/Istanbul
                #2020-01-08 08:30:00(GMT-06:00) Central Standard Time (America/Chicago)
                if re.search(r'.+?\((.+)\)',dispatch_line.group(6)):
                    timezone = re.search(r'.+?\((.+)\)',dispatch_line.group(6)).group(1)
                else:
                    timezone = dispatch_line.group(6)
                ShiftInd , AESTorAEDTtimestamp = isOnShift(timestamp,timezone,before_shift_hours_offset=1)
                if ShiftInd == "On":
                    #print("OnShift {} {} {} {}".format(caseno,timestamp,timezone,line))
                    RawDispatchedEventList.append("-~".join([caseno,severity,AESTorAEDTtimestamp,casetitle]))

    ###### Remove multiple dispatched and keep only the last one ######
    RemoveDuplicateDispatchedEventIdx = []
    RemoveDuplicateDispatchedEventList = []
    ExistingDispatchCaseDic = {} # {'Date':[CaseNoList]}
    
    for idx,line in enumerate(RawDispatchedEventList[::-1]):
        caseno,casesev,casetime,casetitle = line.split('-~')
        date = parser.parse(casetime).strftime("%Y-%m-%d")
        if date not in ExistingDispatchCaseDic.keys():
            ExistingDispatchCaseDic[date] = []
        if caseno not in ExistingDispatchCaseDic[date]:
            ExistingDispatchCaseDic[date].append(caseno)
        else:
            RemoveDuplicateDispatchedEventIdx.append(len(RawDispatchedEventList)-idx-1)
    
    #print(set(RemoveDuplicateDispatchedEventIdx))
    for idx,line in enumerate(RawDispatchedEventList):
        if idx not in set(RemoveDuplicateDispatchedEventIdx):
            RemoveDuplicateDispatchedEventList.append(line)
    
    ###### Sort the Dispatched Case ######
    DispatchedEventRemainingIdx = [i for i in range(len(RemoveDuplicateDispatchedEventList))]
    SortedDispatchedEventIdx = []
    
    for idx in range(len(RemoveDuplicateDispatchedEventList)):
        smallest_idx = DispatchedEventRemainingIdx[0]
        caseno,casesev,smallest_time,casetitle = RemoveDuplicateDispatchedEventList[smallest_idx].split('-~')
        #print("Round {} Setting Smallest {}".format(smallest_idx,casetime1))
        for idx2,line2 in enumerate(RemoveDuplicateDispatchedEventList):
            if idx2 in SortedDispatchedEventIdx:
                continue
            caseno2,casesev2,casetime2,casetitle2 = line2.split('-~')
            if parser.parse(casetime2) < parser.parse(smallest_time):
                #print("Found Smallest {} {}".format(idx2,casetime2))
                smallest_idx = idx2
                smallest_time = casetime2 
        #print("Round {} Smallext_idx {}".format(idx,smallest_idx))
        DispatchedEventRemainingIdx.remove(smallest_idx)
        SortedDispatchedEventIdx.append(smallest_idx)
    
    SortedDispatchedEventList = []
    for idx in SortedDispatchedEventIdx:
        SortedDispatchedEventList.append(RemoveDuplicateDispatchedEventList[idx])
        
    return SortedDispatchedEventList

RawDispatchedEventList = FindDispatchEvent(allevents,debug=True)
RawDispatchedEventList[-10:]

KeyboardInterrupt: 

In [288]:
RawDispatchedEventList[-10:]

['689234940-~3-~2020-06-07 13:30:00-~proactive case || N9K-C93180YC-EX-B || upgrade',
 '689230340-~3-~2020-06-08 10:30:00-~ACI 4.2(4i) -> 3.2(9b):バージョンダウン不可',
 '689241901-~3-~2020-06-09 13:30:00-~TELCEL // APIC-SERVER-L2 // 3.2(6i) // SSH access issue',
 '689201178-~3-~2020-06-10 09:00:00-~I upgraded the CIMC on my APIC, the APIC now the server is hung',
 '689225102-~3-~2020-06-10 13:30:00-~Connection from Palo Alto PA-3020 to Leaf Switch 9k not working for 1Gig Fiber SFP',
 '689243886-~3-~2020-06-10 13:30:00-~ACI 4.2(3j) - Proactive SR for upgrade to 4.2(4i)',
 '689099649-~3-~2020-06-12 10:01:00-~Mainframe Move from Current EPG to App Centric EPG',
 '688771973-~2-~2020-06-14 14:00:00-~N9K-C93180YC-FX // Seeing some false in the ACI environment',
 '689232755-~3-~2020-06-14 14:00:00-~AMUNDI-Proactive SR for ACI-move of the legacy services into the ACI-June 14, 2020',
 '689225102-~3-~2020-06-17 13:00:00-~Connection from Palo Alto PA-3020 to Leaf Switch 9k not working for 1Gig Fiber SFP']

In [296]:
def SortInQueueEventByTime(CaseInQueueList,date='2020-04-05',debug=True):
    
    CaseInQueueIndexSortByTime = []
    CaseInQueueSortedAndRemoveList = []
    CaseInQueueSortedRemaingList = [i for i in range(len(CaseInQueueList))]
    CaseInQueueRemoveDupExtSortByTimeList = []
    CaseInqueueRemoveDateSortByTimeList = []

    shift_hour,gmt_hour = GetShiftHour(date)
    
    for index1, line1 in enumerate(CaseInQueueList):
        
        smallest_caseno,smallest_priority, smallest_queue,smallest_time = CaseInQueueList[CaseInQueueSortedRemaingList[0]].split("-~")
        smallest_datetime = dateutil.parser.parse(smallest_time)
        CaseInQueueIndexSortByTime.append(CaseInQueueSortedRemaingList[0])
        
        for index2, line2 in enumerate(CaseInQueueList[:]):
            
            if index2 in CaseInQueueSortedAndRemoveList:
                #if debug:
                #    print("Skip {} {}".format(index1,index2))
                continue
            caseno2, priority2, queue2, time2 = line2.split("-~")
            datetime2 = dateutil.parser.parse(time2)
            
            #if debug:
                #print("Compare {} {} - {} {}".format(CaseInQueueSortedRemaingList[0],smallest_datetime,index1+index2+1,datetime2))
            
            if datetime2 < smallest_datetime:
                #if debug:
                    #print("Found smaller index{}".format(index2))
                CaseInQueueIndexSortByTime[index1] = index2
                smallest_datetime = datetime2
                
        #if debug:
        #    print("Iteration {} pop {}".format(index1,CaseInQueueIndexSortByTime[index1]))
        CaseInQueueSortedAndRemoveList.append(CaseInQueueIndexSortByTime[index1])
        CaseInQueueSortedRemaingList.remove(CaseInQueueIndexSortByTime[index1])
        
        #if debug:
        #    print(CaseInQueueIndexSortByTime)
        #    print(CaseInQueueSortedRemaingList)
        #    print("Sorted Index from CaseInQueueList {}".format(CaseInQueueIndexSortByTime))

    for index in CaseInQueueIndexSortByTime:
        CaseInQueueRemoveDupExtSortByTimeList.append(CaseInQueueList[index])
    
    if debug:
        print("After Revming Dup Case - to be remoeved DUP FTS/UC")
        for caseline in CaseInQueueRemoveDupExtSortByTimeList:
            print(caseline)
    ###### Remove Dup FTS  List and only keep the last one ######
    ###### Remove Dup UC  List and only keep the last one ######
    ###### Remove Dup Case within 20 seconds and 2 event next to each other and only keep the last one ######
    InQueueFTSList = [] #Index from CaseInQueueRemoveDupExtSortByTimeList #Case , #Timestamp
    InQueueDupFTSIndexList = []
    InQueueUCList = [] #Index from CaseInQueueRemoveDupExtSortByTimeList #Case , #Timestamp
    InQueueDupUCIndexList = []

    LastCaseList = [] #Index from CaseInQueueRemoveDupExtSortByTimeList #Case #Queue #Timestamp
    InQueueDupCaseIndexList = []
    
    ######## Dup case condidition < 1 minutes
    for index, caseline in enumerate(CaseInQueueRemoveDupExtSortByTimeList):
        caseno, priority, queue, casetime = caseline.split("-~")
        if queue == 'FTS':
            InQueueFTSList.append([index,caseno,casetime])
        elif queue == 'UC':
            InQueueUCList.append([index,caseno,casetime])
        else:
            for LastCase in LastCaseList:
                if LastCase and LastCase[1] == caseno and LastCase[3] == queue:
                    case_datetime = parser.parse(casetime)
                    last_datetime = parser.parse(LastCase[4])
                    ######## Dup case condition ######
                    #1. same events come in < 60 seconds, 2 events in a row 
                    #2. priority are different , 2 events in a row
                    if (case_datetime - last_datetime).seconds <= 60 or LastCase[2] != priority:
                        if debug:
                            print("Found Dup Case {} {} {}".format(LastCase[0],caseno,LastCase[4]))
                        InQueueDupCaseIndexList.append(LastCase[0])
            LastCaseList.append([index,caseno,priority,queue,casetime])
    InQueueDupCaseIndexList = set(InQueueDupCaseIndexList)
    
    ###### Get InQueueDupFTSIndexList in order ######
    for index1, fts_case1 in enumerate(InQueueFTSList):
        for index2, fts_case2 in enumerate(InQueueFTSList[index1+1:]):
            if fts_case1[1] == fts_case2[1]:
                if debug:
                    print("Dup FTS index {} with {} and remove {}".format(fts_case1[0],fts_case2[0],fts_case1[0]))
                InQueueDupFTSIndexList.append(fts_case1[0])
    ###### Get InQueueDupUCIndexList in order ######
    for index1, uc_case1 in enumerate(InQueueUCList):
        for index2, uc_case2 in enumerate(InQueueUCList[index1+1:]):
            if uc_case1[1] == uc_case2[1]:
                if debug:
                    print("Dup UC index {} with {} and remove {}".format(uc_case1[0],uc_case2[0],uc_case1[0]))
                InQueueDupUCIndexList.append(uc_case1[0])
    if debug:
        print("Dup Case List {}".format(" ".join([str(index) for index in InQueueDupCaseIndexList])))
        print("Dup FTS Case List {}".format(" ".join([str(index) for index in set(InQueueDupFTSIndexList)])))
        print("Dup UC Case List {}".format(" ".join([str(index) for index in set(InQueueDupUCIndexList)])))
        
    InQueueDupIndexList = sorted([*InQueueDupCaseIndexList,*set(InQueueDupFTSIndexList),*set(InQueueDupUCIndexList)])
    
    for index in InQueueDupIndexList[::-1]:
        CaseInQueueRemoveDupExtSortByTimeList.pop(index)
        
    ###### Remove Case which is not for that date ######
    for caseline in CaseInQueueRemoveDupExtSortByTimeList:
        caseno, priority, queue, time = caseline.split("-~")
        line_datetime = parser.parse(time)
        if line_datetime.strftime("%Y-%m-%d") == date:
            CaseInqueueRemoveDateSortByTimeList.append(caseline)

    return CaseInqueueRemoveDateSortByTimeList

InQueueEventInOrderList = SortInQueueEventByTime(InQueueEventList,date=date,debug=True)
InQueueEventInOrderList

After Revming Dup Case - to be remoeved DUP FTS/UC
689238498-~3-~EME-ACI-Solutions-~2020-06-08 06:01:13
689239839-~3-~EME-ACI-Solutions-~2020-06-08 06:02:42
689154452-~1-~FTS-~2020-06-08 06:05:10
689154452-~1-~FTS-~2020-06-08 06:05:10
689233408-~3-~EME-ACI-Solutions-~2020-06-08 06:21:31
689233408-~3-~EME-ACI-Solutions-~2020-06-08 06:21:37
689239934-~3-~EME-ACI-Solutions-~2020-06-08 06:25:57
689185699-~3-~EME-ACI-Solutions-~2020-06-08 06:42:03
689185699-~3-~EME-ACI-Solutions-~2020-06-08 06:42:10
689240082-~1-~EME-ACI-Solutions-~2020-06-08 06:55:41
689240082-~1-~EME-ACI-Solutions-~2020-06-08 07:24:56
689224906-~3-~FTS-~2020-06-08 07:28:48
689224906-~3-~FTS-~2020-06-08 07:28:48
689224906-~3-~FTS-~2020-06-08 07:28:48
689240783-~3-~EME-ACI-Solutions-~2020-06-08 08:45:19
689183304-~3-~UC-~2020-06-08 09:11:04
689183304-~3-~UC-~2020-06-08 09:11:04
689241006-~3-~EME-ACI-Solutions-~2020-06-08 09:13:54
689241320-~3-~EME-ACI-Solutions-~2020-06-08 10:05:34
689241375-~3-~EME-ACI-Solutions-~2020-06-0

['689238498-~3-~EME-ACI-Solutions-~2020-06-08 06:01:13',
 '689239839-~3-~EME-ACI-Solutions-~2020-06-08 06:02:42',
 '689154452-~1-~FTS-~2020-06-08 06:05:10',
 '689239934-~3-~EME-ACI-Solutions-~2020-06-08 06:25:57',
 '689185699-~3-~EME-ACI-Solutions-~2020-06-08 06:42:10',
 '689240082-~1-~EME-ACI-Solutions-~2020-06-08 06:55:41',
 '689240082-~1-~EME-ACI-Solutions-~2020-06-08 07:24:56',
 '689224906-~3-~FTS-~2020-06-08 07:28:48',
 '689240783-~3-~EME-ACI-Solutions-~2020-06-08 08:45:19',
 '689183304-~3-~UC-~2020-06-08 09:11:04',
 '689241006-~3-~EME-ACI-Solutions-~2020-06-08 09:13:54',
 '689241320-~3-~EME-ACI-Solutions-~2020-06-08 10:05:34',
 '689241375-~3-~EME-ACI-Solutions-~2020-06-08 10:15:01',
 '689241675-~3-~EME-ACI-Solutions-~2020-06-08 11:07:06',
 '689182716-~2-~EME-ACI-Solutions-~2020-06-08 11:11:28',
 '689233408-~2-~EME-ACI-Solutions-~2020-06-08 11:59:13']

In [155]:
CaseStatsDic = {"SydQueueVol":0,"BGLQueueVol":0,
            "SydTakenVol":0,"BGLTakenVol":0,
            'FTSCaseVol':0,'UCCaseVol':0,
            "SydQueueVolByPri":[0,0,0,0,0,0],"BGLQueueVolByPri":[0,0,0,0,0,0], #0 FTS #1,2,3,4 Priority #5 Urgent Collab
            "SydTakenVolByPri":[0,0,0,0,0,0],"BGLTakenVolByPri":[0,0,0,0,0,0], #0 FTS #1,2,3,4 Priority #5 Urgent Collab
            "SydOnShiftEngr":1,"BGLOnShiftEngr":1,
            "SydOnShiftEngrTakenVol":0,"BGLOnShiftEngrTakenVol":1,
            "SYDHelper":0,"BGLHelper":0,
            }

def GetStatsFromInQueueEventInOrderList(InQueueEventInOrderList,CaseStatsDic):
    
    ###### Getting partial CaseStatsDic Stats from the orderred List ######
    for caseline in InQueueEventInOrderList:
        caseno, priority, queue, time = caseline.split("-~")
        if 'SYD' in queue:
            CaseStatsDic['SydQueueVol'] = CaseStatsDic['SydQueueVol'] + 1
            CaseStatsDic['SydQueueVolByPri'][int(priority)] = CaseStatsDic['SydQueueVolByPri'][int(priority)]+1
        elif 'BLR' in queue:
            CaseStatsDic['BGLQueueVol'] = CaseStatsDic['BGLQueueVol'] + 1
            CaseStatsDic['BGLQueueVolByPri'][int(priority)] = CaseStatsDic['BGLQueueVolByPri'][int(priority)]+1
        elif 'FTS' in queue:
            CaseStatsDic['FTSCaseVol'] = CaseStatsDic['FTSCaseVol'] + 1
        elif 'UC' in queue:
            CaseStatsDic['UCCaseVol'] = CaseStatsDic['UCCaseVol'] + 1
    
    return CaseStatsDic

CaseStatsDic = GetStatsFromInQueueEventInOrderList(InQueueEventInOrderList,CaseStatsDic)
CaseStatsDic

{'SydQueueVol': 0,
 'BGLQueueVol': 0,
 'SydTakenVol': 0,
 'BGLTakenVol': 0,
 'FTSCaseVol': 1,
 'UCCaseVol': 0,
 'SydQueueVolByPri': [0, 0, 0, 0, 0, 0],
 'BGLQueueVolByPri': [0, 0, 0, 0, 0, 0],
 'SydTakenVolByPri': [0, 0, 0, 0, 0, 0],
 'BGLTakenVolByPri': [0, 0, 0, 0, 0, 0],
 'SydOnShiftEngr': 1,
 'BGLOnShiftEngr': 1,
 'SydOnShiftEngrTakenVol': 0,
 'BGLOnShiftEngrTakenVol': 1,
 'SYDHelper': 0,
 'BGLHelper': 0}

In [179]:
#/app/RabbitMQ_TAC_Event_Handler.py-2020-04-18 11:23:26,480-INFO-MQToRedis:688881426 3 Case_Accepted Chileaf402 -High SSD usage observed- Equipment Flash Warning taken by dashmari HTTS-EMEAR-DC-ACI at 2020-04-18 11:21:24

def CaseAcceptInOrderByTime(AllEventsList,date='2020-02-27',debug=False):

    shift_hour,gmt_hour = GetShiftHour(date)
    
    #1 Case No #2 Severity #3 CaseAccept Type Case|FTS|UC #4 ccoid #5 Workgroup #6 Time
    Accept_Reg = re.compile(r'.*?MQToRedis:(\d{9})\s(\d)\s(Case|FTS|UC)_Accepted.*\sby\s(\w+)\s(.*)\s{0,1}at\s(\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2})')
    UC_Closed_Reg = re.compile(r'.*?MQToRedis:(\d{9})\s(\d)\sUC_Closed.*\sby\s(\w+)\s(.*)\s{0,1}at\s(\d{4}-\d{2}-\d{2}\s\d{2}:\d{2}:\d{2})')
    
    New_AllEventsList = []
    for event in AllEventsList:
        if 'Exception' not in event:
            New_AllEventsList.append(event)
            
    CaseRawAcceptList = []
    RawUCClosedList = []
    
    for line in AllEventsList:
        if re.search(r'_Accepted',line):
            result = re.search(Accept_Reg,line)
            casetype = result.group(3).strip()
            workgroup = result.group(5).strip()
            if not workgroup:
                workgroup = "Unknown"
            if casetype == 'FTS':
                workgroup = 'FTS_WORKGROUP'
            elif casetype == 'UC':
                workgroup = 'UC_WORKGROUP'
            #print("{} {} {}".format(line,casetype,workgroup))
            CaseRawAcceptList.append(result.group(1)+"-~"+result.group(2)+"-~"+result.group(4)+"-~"+
                                    workgroup+"-~"+result.group(6))
        elif re.search(r'UC_Closed',line):
            result = re.search(UC_Closed_Reg,line)
            RawUCClosedList.append(result.group(1)+"-~"+result.group(2)+"-~"+result.group(3)+"-~"+
                                    result.group(4)+"-~"+result.group(5))
    
    #CaseRawAcceptList = [re.search(Accept_Reg,line).group(1)+"-~"+
    #                     re.search(Accept_Reg,line).group(2)+"-~"+
    #                     re.search(Accept_Reg,line).group(4)+"-~"+
    #                     re.search(Accept_Reg,line).group(5)+"-~"+
    #                     re.search(Accept_Reg,line).group(6)
    #                     for line in New_AllEventsList if re.search(r'_Accepted',line)]
    
    #print("CasAcceptInOrderByTime:Raw Accept List Length {} UC_Closed_List {}".format(len(CaseRawAcceptList),len(RawUCClosedList)))
    
    ###### Handling Accept Case ######
    DuplicateAcceptIndexList = []
    for index1, raw_case_line1 in enumerate(CaseRawAcceptList):
        caseno1, severity1, queue1, workgroup, time1 = raw_case_line1.split("-~")
        for index2, raw_case_line2 in enumerate(CaseRawAcceptList[index1+1:]):
            caseno2, severity2, queue2, workgroup, time2 = raw_case_line2.split("-~")
            if debug:
                pass
                #print("CasAcceptInOrderByTime:Comparing {} {} {} - {} {} {}".format(caseno1,queue1,time1,caseno2,queue2,time2))
            if caseno1 == caseno2 and time1 == time2:
                if debug:
                    print("Found dupliate {} {}".format(index1,index1+index2+1))
                DuplicateAcceptIndexList.append(index1+index2+1)
    DuplicateAcceptIndexList = sorted(list(set(DuplicateAcceptIndexList)))
    #print("CasAcceptInOrderByTime:Duplicate Case Index list {} {}".format(len(DuplicateAcceptIndexList),DuplicateAcceptIndexList))
    CaseAcceptRemoveDupExtList = [line for line in CaseRawAcceptList]
    for index in DuplicateAcceptIndexList[::-1]:
        #print("CasAcceptInOrderByTime:Pop duplicate {} from list".format(index))
        CaseAcceptRemoveDupExtList.pop(index)
        #print("CasAcceptInOrderByTime:After removing DuplicateCase length {}".format(len(CaseAcceptRemoveDupExtList)))
    ###### Remove the case Accept time out of the current date ######
    CaseAcceptRemoveDupExtListWithDate = []
    for caseline in CaseAcceptRemoveDupExtList:
        #print(caseline)
        caseno, severity, ccoid, workgroup, casetime = caseline.split("-~")
        line_datetime = parser.parse(casetime)
        if line_datetime.strftime("%Y-%m-%d") == date:
            CaseAcceptRemoveDupExtListWithDate.append(caseline)
    UniqueAcceptEventInOrderList = []
    for line in CaseAcceptRemoveDupExtListWithDate[::-1]:
        caseno, severity, ccoid, workgroup, timestamp = line.split('-~')
        isFoundDupCase = False
        for uniline in UniqueAcceptEventInOrderList[::-1]:
            unicaseno, severity, uniccoid, workgroup, unitimestamp = uniline.split('-~')
            if (unicaseno == caseno) and (uniccoid == ccoid):
                isFoundDupCase = True
                break
        if not isFoundDupCase:
            UniqueAcceptEventInOrderList.append(line) 
            
    ##### Handling UC_Closed Case ######
    DuplicateUCClosedIndexList = []
    for index1, raw_case_line1 in enumerate(RawUCClosedList):
        caseno1, severity1, ccoid1, workgroup1, time1 = raw_case_line1.split("-~")
        for index2, raw_case_line2 in enumerate(RawUCClosedList[index1+1:]):
            caseno2, severity2, ccoid2, workgroup2, time2 = raw_case_line2.split("-~")
            #print("CasAcceptInOrderByTime:Comparing {} {} {} - {} {} {}".format(caseno1,queue1,time1,caseno2,queue2,time2))
            if caseno1 == caseno2 and time1 == time2:
                #print("Found dupliate {} {}".format(index1,index1+index2+1))
                #DuplicateUCClosedIndexList.append(index1+index2+1)
                DuplicateUCClosedIndexList.append(index1)
    DuplicateUCClosedIndexList = sorted(list(set(DuplicateUCClosedIndexList)))
    #print("CaseUC_Cosed:Duplicate Closed Index list {} {}".format(len(DuplicateUCClosedIndexList),DuplicateUCClosedIndexList))
    CaseUCCLosedRemoveDupExtList = [line for line in RawUCClosedList]
    for index in DuplicateUCClosedIndexList[::-1]:
        #print("CasAcceptInOrderByTime:Pop duplicate {} from list".format(index))
        CaseUCCLosedRemoveDupExtList.pop(index)
        #print("CasAcceptInOrderByTime:After removing DuplicateCase length {}".format(len(CaseAcceptRemoveDupExtList)))
    ###### Remove the case UC_Closed time out of the current date ######
    CaseUCClosedRemoveDupExtListWithDate = []
    for caseline in CaseUCCLosedRemoveDupExtList:
        caseno, severity, ccoid, workgroup, casetime = caseline.split("-~")
        line_datetime = parser.parse(casetime)
        if line_datetime.strftime("%Y-%m-%d") == date:
            CaseUCClosedRemoveDupExtListWithDate.append(caseline)
            
    ###### From UniqueAcceptEventInOrderList remove UC_Closed by CaseUCClosedRemoveDupExtListWithDate ######
    CaseAcceptRemoveUCClosedList = []
    CaseAcceptRemoveUCClosedIndexList = []
    for idx1,line1 in enumerate(UniqueAcceptEventInOrderList):
        caseno1,casesev1,ccoid1,workgroup1,timestamp1 = line1.split("-~")
        for idx2,line2 in enumerate(CaseUCCLosedRemoveDupExtList):
            caseno2,casesev2,ccoid2,workgroup2,timestamp2 = line2.split("-~")
            if caseno1==caseno2 and ccoid1==ccoid2:
                print("Found UC_Closed {} in Accept {}".format(idx2,idx1))
                CaseAcceptRemoveUCClosedIndexList.append(idx1)
    CaseAcceptRemoveUCClosedList = [line for idx,line in enumerate(UniqueAcceptEventInOrderList) if idx not in CaseAcceptRemoveUCClosedIndexList]
        
    return CaseAcceptRemoveUCClosedList[::-1]

AcceptEventInOrderList = CaseAcceptInOrderByTime(allevents_in_date,date='2020-05-25',debug=False)
AcceptEventInOrderList

Found UC_Closed 0 in Accept 1
['689144972-~4-~shokada-~JAPAN-NEXUS-~2020-05-25 00:44:41', '689150498-~2-~junwa-~FTS_WORKGROUP-~2020-05-25 00:32:44', '689150965-~2-~afaizur-~HTTS-APAC-ENT1-~2020-05-25 00:02:19', '689144570-~4-~ykamimur-~JAPAN-ACI-~2020-05-25 00:00:44']


['689144570-~4-~ykamimur-~JAPAN-ACI-~2020-05-25 00:00:44',
 '689150965-~2-~afaizur-~HTTS-APAC-ENT1-~2020-05-25 00:02:19',
 '689150498-~2-~junwa-~FTS_WORKGROUP-~2020-05-25 00:32:44',
 '689144972-~4-~shokada-~JAPAN-NEXUS-~2020-05-25 00:44:41']

In [357]:
SYDEngrList = ['tonzeng','minkwong','wilchong','debabbar','lindawa','junwa']
SYDOtherEngrList = ['zdazhi','zmeng']
BGLEngrList = ['deepaky','knagavol','jawalia','prpratee','maveer','raghb','roagraw2','shparanj','vkalmath']
BGLOtherEngrList = ['vnandan','anirukas','reperuma','deepakba','bharatkc','kdoodi']

def UpdateEngrListFromAcceptEventInOrderList(AcceptEventInOrderList,EngrList,WKGroupList=['APT-ACI-SOLUTIONS','APT-ACI-SOLUTIONS2']):
    
    SYDEngrList = EngrList[0]
    SYDOtherEngrList = EngrList[1]
    BGLEngrList = EngrList[2]
    BGLOtherEngrList = EngrList[3]
    
    Syd_group_Nname = WKGroupList[0]
    BLR_group_name =WKGroupList[1]
    
    for caseline in AcceptEventInOrderList:
        caseno, severity, ccoid, workgroup, time = caseline.split("-~")
        if workgroup == "Unknown":
            continue
        if workgroup == Syd_group_Nname:
            if ccoid not in SYDEngrList:
                SYDEngrList.append(ccoid)
                print('Updating {} in SydEngrList'.format(ccoid))
        elif workgroup == BLR_group_name:
            if ccoid not in BGLEngrList:
                print('Updating {} in BGLEngrList'.format(ccoid))
                BGLEngrList.append(ccoid)
                if ccoid in BGLOtherEngrList:
                    print('Removing {} from BGLOtherEngrList'.format(ccoid))
                    BGLOtherEngrList.remove(ccoid)
            elif ccoid in BGLOtherEngrList:
                print('Updating {} in BGLOtherEngrList'.format(ccoid))
                if ccoid in BGLEngrList:
                    print('Removing {} from BGLEngrList'.format(ccoid))
                    BGLEngrList.remove(ccoid)
                    
    return [SYDEngrList,SYDOtherEngrList,BGLEngrList,BGLOtherEngrList]
                    
SYDEngrList,SYDOtherEngrList,BGLEngrList,BGLOtherEngrList = UpdateEngrListFromAcceptEventInOrderList(AcceptEventInOrderList,[SYDEngrList,SYDOtherEngrList,BGLEngrList,BGLOtherEngrList])

SYDEngrList,SYDOtherEngrList,BGLEngrList, BGLOtherEngrList

(['tonzeng', 'minkwong', 'wilchong', 'debabbar', 'lindawa', 'junwa'],
 ['zdazhi', 'zmeng'],
 ['deepaky',
  'knagavol',
  'jawalia',
  'prpratee',
  'maveer',
  'raghb',
  'roagraw2',
  'shparanj',
  'vkalmath'],
 ['vnandan', 'anirukas', 'reperuma', 'deepakba', 'bharatkc', 'kdoodi'])

In [358]:
###### Case Accept Stats ######
def GetStatsFromCaseAcceptInOrderByTime(AcceptEventInOrderList,CaseStatsDic,EngrList):
    
    SYDEngrList = EngrList[0]
    SYDOtherEngrList = EngrList[1]
    BGLEngrList = EngrList[2]
    BGLOtherEngrList = EngrList[3]
        
    SydOnShiftEngrList = []
    SydOtherOnshiftEngrList = []

    BGLOnShiftEngrList = []
    BGLOtherOnshiftEngrList = []
    
    ###### Getting partial CaseStatsDic Stats from the orderred List ######
    for caseline in AcceptEventInOrderList:
        caseno, severity, ccoid, workgroup, time = caseline.split("-~")
        if ccoid in SYDEngrList:
            if ccoid not in SydOnShiftEngrList:
                SydOnShiftEngrList.append(ccoid)
        elif ccoid in SYDOtherEngrList:
            if ccoid not in SydOtherOnshiftEngrList:
                SydOtherOnshiftEngrList.append(ccoid)
        elif ccoid in BGLEngrList:
            if ccoid not in BGLOnShiftEngrList:
                BGLOnShiftEngrList.append(ccoid)
        elif ccoid in BGLOtherEngrList:
            if ccoid not in BGLOtherOnshiftEngrList:
                BGLOtherOnshiftEngrList.append(ccoid)
                
    CaseStatsDic['SydOnShiftEngr'] = SydOnShiftEngrList
    CaseStatsDic['BGLOnShiftEngr'] = BGLOnShiftEngrList
    CaseStatsDic['SYDHelper'] = SydOtherOnshiftEngrList
    CaseStatsDic['BGLHelper'] = BGLOtherOnshiftEngrList
    
    return CaseStatsDic

CaseStatsDic = GetStatsFromCaseAcceptInOrderByTime(AcceptEventInOrderList,CaseStatsDic,[SYDEngrList,SYDOtherEngrList,BGLEngrList,BGLOtherEngrList])
CaseStatsDic

{'SydQueueVol': 0,
 'BGLQueueVol': 0,
 'SydTakenVol': 0,
 'BGLTakenVol': 0,
 'FTSCaseVol': 0,
 'UCCaseVol': 0,
 'SydQueueVolByPri': [0, 0, 0, 0, 0, 0],
 'BGLQueueVolByPri': [0, 0, 0, 0, 0, 0],
 'SydTakenVolByPri': [0, 0, 0, 0, 0, 0],
 'BGLTakenVolByPri': [0, 0, 0, 0, 0, 0],
 'SydOnShiftEngr': [],
 'BGLOnShiftEngr': [],
 'SydOnShiftEngrTakenVol': 0,
 'BGLOnShiftEngrTakenVol': 1,
 'SYDHelper': [],
 'BGLHelper': ['vnandan']}

In [359]:
CaseTakenByEngrDic = {}

for case in AcceptEventInOrderList:
    
    caseno,severity,ccoid,workgroup,time = case.split("-~")
    if ccoid not in CaseTakenDic.keys():
        CaseTakenDic[ccoid] = []
    if caseno not in CaseTakenDic[ccoid]:
        CaseTakenDic[ccoid].append(caseno)
    if ccoid not in OnShiftEngrList and (ccoid in SYDEngrList or ccoid in BGLEngrList 
                                         or ccoid in BGLOtherEngrList or ccoid in SYDOtherEngrList):
        OnShiftEngrList.append(ccoid)
    if caseno not in CaseTakenByEngrDic.keys():
        CaseTakenByEngrDic[caseno] = []
        
    if ccoid in SYDEngrList or ccoid in SYDOtherEngrList:
        CaseStatsDic['SydTakenVolByPri'][int(severity)] = CaseStatsDic['SydTakenVolByPri'][int(severity)] + 1
        CaseTakenByEngrDic[caseno].append(ccoid+"(SYD)")
    elif ccoid in BGLEngrList or ccoid in BGLOtherEngrList:
        CaseStatsDic['BGLTakenVolByPri'][int(severity)] = CaseStatsDic['BGLTakenVolByPri'][int(severity)] + 1
        CaseTakenByEngrDic[caseno].append(ccoid+"(BLR)")
    elif ccoid == 'lowtouch':
        CaseTakenByEngrDic[caseno].append(ccoid+"(HW)")
    else:
        CaseTakenByEngrDic[caseno].append(ccoid+"(Unknown)")
        
CaseTakenByEngrDic,CaseStatsDic

({'689085612': ['jampatte(Unknown)'],
  '689085611': ['ankisha6(Unknown)'],
  '689082013': ['qiuxia(Unknown)'],
  '688966607': ['nagordon(Unknown)'],
  '689085663': ['ananyraj(Unknown)'],
  '689085694': ['anahar(Unknown)'],
  '689085747': ['aharalal(Unknown)'],
  '689085768': ['rongzli(Unknown)'],
  '689066852': ['pt2(Unknown)'],
  '689072536': ['sankbhat(Unknown)'],
  '689085804': ['jingjjin(Unknown)'],
  '689085824': ['caiswang(Unknown)'],
  '689076160': ['amraja(Unknown)'],
  '689085916': ['jiaqfeng(Unknown)'],
  '689078597': ['caiswang(Unknown)'],
  '689085926': ['frjin(Unknown)'],
  '689085988': ['jinglwu(Unknown)'],
  '688973823': ['ranavada(Unknown)'],
  '689086027': ['rongzli(Unknown)'],
  '689086040': ['deeptipa(Unknown)'],
  '689086017': ['jiusong(Unknown)'],
  '689086115': ['lowtouch(HW)', 'rakusman(Unknown)'],
  '689086099': ['krajasel(Unknown)'],
  '689081733': ['aharalal(Unknown)'],
  '689086179': ['jiusong(Unknown)'],
  '689086213': ['pt2(Unknown)'],
  '689086238': ['man

In [360]:
for ccoid, caselist in CaseTakenDic.items():
    if ccoid in SYDEngrList:
        CaseTakenDicInclNo['SYD'][ccoid] = list([len(caselist),*caselist])
    elif ccoid in SYDOtherEngrList:
        CaseTakenDicInclNo['SYD_Other'][ccoid] = list([len(caselist),*caselist])
    elif ccoid in BGLEngrList:
        CaseTakenDicInclNo['BGL'][ccoid] = list([len(caselist),*caselist])
    elif ccoid in BGLOtherEngrList:
        CaseTakenDicInclNo['BGL_Other'][ccoid] = list([len(caselist),*caselist])
CaseTakenDicInclNo

{'SYD': {},
 'SYD_Other': {},
 'BGL': {},
 'BGL_Other': {'vnandan': [1, '689081281']}}

In [253]:
shift_hour,gmt_hour = ACILib.GetShiftHour(date)
print(date)
print(shift_hour)
print(gmt_hour)

InQueueEventInOrderListWithEngr = []
for caseline in InQueueEventInOrderList:
    caseno,priority,queue,timestamp = caseline.split("-~")
    ###### CaseStatsDic FTS Stats ######
    if queue == 'FTS':
        for ccoid,caselist in CaseTakenDic.items():
            if caseno in caselist:
                if ccoid in SYDEngrList or ccoid in SYDOtherEngrList:
                    CaseStatsDic['SydTakenVolByPri'][0] = CaseStatsDic['SydTakenVolByPri'][0] + 1
                    #CaseStatsDic['SydTakenVol'] = CaseStatsDic['SydTakenVol'] + 1
                elif ccoid in BGLEngrList or ccoid in BGLOtherEngrList:
                    CaseStatsDic['BGLTakenVolByPri'][0] = CaseStatsDic['BGLTakenVolByPri'][0] + 1
                    #CaseStatsDic['BGLTakenVol'] = CaseStatsDic['BGLTakenVol'] + 1
    
    ###### CaseStatsDic UC Stats ######
    elif queue == 'UC':
        for ccoid,caselist in CaseTakenDic.items():
            if caseno in caselist:
                if ccoid in SYDEngrList or ccoid in SYDOtherEngrList:
                    CaseStatsDic['SydTakenVolByPri'][5] = CaseStatsDic['SydTakenVolByPri'][5] + 1
                    #CaseStatsDic['SydTakenVol'] = CaseStatsDic['SydTakenVol'] + 1
                elif ccoid in BGLEngrList or ccoid in BGLOtherEngrList:
                    CaseStatsDic['BGLTakenVolByPri'][5] = CaseStatsDic['BGLTakenVolByPri'][5] + 1   
                    #CaseStatsDic['BGLTakenVol'] = CaseStatsDic['BGLTakenVol'] + 1
    
    if caseno in CaseTakenByEngrDic.keys():
        InQueueEventInOrderListWithEngr.append(caseno+'-~'+priority+"-~"+queue+'-~'+
                                               (parser.parse(timestamp)+datetime.timedelta(hours=(gmt_hour+shift_hour[0]))).strftime("%Y-%m-%d %H:%M:%S")+
                                               '-~'+"-~".join(CaseTakenByEngrDic[caseno]))
    else:
        InQueueEventInOrderListWithEngr.append(caseno+'-~'+priority+"-~"+queue+'-~'+
                                               (parser.parse(timestamp)+datetime.timedelta(hours=(gmt_hour+shift_hour[0]))).strftime("%Y-%m-%d %H:%M:%S")
                                               )
InQueueEventInOrderListWithEngr

ShiftStartHourInGMT 0 GMT 10
2020-05-08
[0, 6]
10


['689046543-~1-~CX-APJC-SYD-ACI-SSPT-~2020-05-08 10:02:17-~siddhp(SYD)',
 '689026455-~3-~FTS-~2020-05-08 10:11:14-~maveer(BLR)',
 '689023549-~3-~FTS-~2020-05-08 10:11:28-~minkwong(SYD)',
 '689046526-~2-~CX-APJC-BLR-ACI-SSPT-~2020-05-08 10:12:14-~jawalia(BLR)',
 '689046626-~3-~CX-APJC-SYD-ACI-SSPT-~2020-05-08 10:27:03-~debabbar(SYD)-~srmb(Unknown)',
 '689044109-~3-~UC-~2020-05-08 11:10:57-~zdazhi(SYD)',
 '689046822-~3-~WW-ACI-Solutions-~2020-05-08 11:26:37-~minkwong(SYD)',
 '689046797-~3-~WW-ACI-Solutions-~2020-05-08 11:34:59-~kahande(Unknown)',
 '689046863-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-08 11:35:24-~deepaky(BLR)',
 '689046920-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-08 11:50:15-~reperuma(BLR)',
 '689046948-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-08 12:00:14-~hethakur(Unknown)',
 '689047028-~3-~CX-APJC-SYD-ACI-SSPT-~2020-05-08 12:24:29-~debabbar(SYD)',
 '689025434-~3-~CX-APJC-BLR-ACI-SSPT-~2020-05-08 12:30:16-~raghb(BLR)',
 '689028908-~2-~CX-APJC-BLR-ACI-SSPT-~2020-05-08 12:40:04-~maveer(BLR)',
 

In [311]:
import glob
###### InQueue file parsing Read the current Parsing file 2020_InQueue.txt ######
jupyter_container_path = '/home/jovyan'
flask_container_path = ''
current_container_path = jupyter_container_path
inqueuefilenames = current_container_path + '/HTTSDashboard/logs/TAC/EMEA/ACI/events/*_InQueueEvent.txt'
inqueuefilelist = glob.glob(inqueuefilenames)

def FindInQueueEventsFromEventFile(inqueuefilelist):
    if len(inqueuefilelist) == 0:
        return
    InQueueEvents_by_year = {}
    for inqueuefile in inqueuefilelist:
        with open(inqueuefile,'r') as f:
            ReadInQueueEvents = f.readlines()
        f.close()
        ReadInQueueEvents = [event.strip() for event in ReadInQueueEvents if event.strip()]
        for event in ReadInQueueEvents:
            caseno, casesev, queue , timestamp = event.split("-~")
            caseyear = parser.parse(timestamp).year
            if caseyear not in InQueueEvents_by_year.keys():
                InQueueEvents_by_year[caseyear] = []
            InQueueEvents_by_year[caseyear].append(event)
    return InQueueEvents_by_year

InQueueEvents_by_year = FindInQueueEventsFromEventFile(inqueuefilelist)
InQueueEvents_by_year[2020][-20:]

['689212170-~3-~EME-ACI-Solutions-~2020-06-09 06:07:22',
 '689154452-~1-~UC-~2020-06-09 06:11:02',
 '689232009-~3-~EME-ACI-Solutions-~2020-06-09 06:15:19',
 '689248990-~3-~EME-ACI-Solutions-~2020-06-09 06:16:15',
 '689238498-~3-~EME-ACI-Solutions-~2020-06-08 06:01:13',
 '689239839-~3-~EME-ACI-Solutions-~2020-06-08 06:02:42',
 '689154452-~1-~FTS-~2020-06-08 06:05:10',
 '689239934-~3-~EME-ACI-Solutions-~2020-06-08 06:25:57',
 '689185699-~3-~EME-ACI-Solutions-~2020-06-08 06:42:10',
 '689240082-~1-~EME-ACI-Solutions-~2020-06-08 06:55:41',
 '689240082-~1-~EME-ACI-Solutions-~2020-06-08 07:24:56',
 '689224906-~3-~FTS-~2020-06-08 07:28:48',
 '689240783-~3-~EME-ACI-Solutions-~2020-06-08 08:45:19',
 '689183304-~3-~UC-~2020-06-08 09:11:04',
 '689241006-~3-~EME-ACI-Solutions-~2020-06-08 09:13:54',
 '689241320-~3-~EME-ACI-Solutions-~2020-06-08 10:05:34',
 '689241375-~3-~EME-ACI-Solutions-~2020-06-08 10:15:01',
 '689241675-~3-~EME-ACI-Solutions-~2020-06-08 11:07:06',
 '689182716-~2-~EME-ACI-Solution

In [326]:
######Function to remove a speicifc date inqueue events ######
def RemoveInQueueEventByDateFromFile(date,InQueueEvents):
    if not InQueueEvents:
        return {}
    RemovedDateInQueueEvents = copy.deepcopy(InQueueEvents)
    for year,events in InQueueEvents.items():
        for index,event in enumerate(InQueueEvents[year][::-1]):
            caseno,severity,queue,casetime = event.split('-~')
            if parser.parse(casetime).strftime("%Y-%m-%d") == date:
                #print("Remove InQueueEvents {} - {}".format(date,event.strip()))
                RemovedDateInQueueEvents[year].pop(len(InQueueEvents[year])-index-1)
            if parser.parse(casetime) < parser.parse(date):
                break
    return RemovedDateInQueueEvents

date = (datetime.datetime.today() - datetime.timedelta(days=0)).strftime("%Y-%m-%d")
RemovedInQueueEvents = RemoveInQueueEventByDateFromFile(date,InQueueEvents_by_year)
RemovedInQueueEvents[2020] = RemovedInQueueEvents[2020][0:5]
RemovedInQueueEvents

{2020: ['689154452-~1-~FTS-~2020-06-09 06:07:20',
  '689212170-~3-~EME-ACI-Solutions-~2020-06-09 06:07:22',
  '689154452-~1-~UC-~2020-06-09 06:11:02',
  '689232009-~3-~EME-ACI-Solutions-~2020-06-09 06:15:19',
  '689248990-~3-~EME-ACI-Solutions-~2020-06-09 06:16:15']}

In [322]:
AddEventList = InQueueEventInOrderList
AddEventList

['689238498-~3-~EME-ACI-Solutions-~2020-06-08 06:01:13',
 '689239839-~3-~EME-ACI-Solutions-~2020-06-08 06:02:42',
 '689154452-~1-~FTS-~2020-06-08 06:05:10',
 '689239934-~3-~EME-ACI-Solutions-~2020-06-08 06:25:57',
 '689185699-~3-~EME-ACI-Solutions-~2020-06-08 06:42:10',
 '689240082-~1-~EME-ACI-Solutions-~2020-06-08 06:55:41',
 '689240082-~1-~EME-ACI-Solutions-~2020-06-08 07:24:56',
 '689224906-~3-~FTS-~2020-06-08 07:28:48',
 '689240783-~3-~EME-ACI-Solutions-~2020-06-08 08:45:19',
 '689183304-~3-~UC-~2020-06-08 09:11:04',
 '689241006-~3-~EME-ACI-Solutions-~2020-06-08 09:13:54',
 '689241320-~3-~EME-ACI-Solutions-~2020-06-08 10:05:34',
 '689241375-~3-~EME-ACI-Solutions-~2020-06-08 10:15:01',
 '689241675-~3-~EME-ACI-Solutions-~2020-06-08 11:07:06',
 '689182716-~2-~EME-ACI-Solutions-~2020-06-08 11:11:28',
 '689233408-~2-~EME-ACI-Solutions-~2020-06-08 11:59:13']

In [329]:
######Function to add a speicifc date inqueue inorder events ######
def AddInQueueEventByDateFromFile(date,AddList,InQueueEvents):
    
    #print("InQueuEvents in func AddInQueueEventByDateFromFile {}".format(InQueueEvents))
    if not InQueueEvents:
        InQueueEvents = {}
    if parser.parse(date).year not in InQueueEvents.keys():
        InQueueEvents[parser.parse(date).year] = []
    ######Need to deep copy the list ######
    AddedDateInQueueEvents = copy.deepcopy(InQueueEvents)
    insert_index = len(InQueueEvents[parser.parse(date).year])

    print("initialize insert_index {}".format(len(InQueueEvents[parser.parse(date).year])))
    for year, events in InQueueEvents.items():
        for index,event in enumerate(events[::-1]):
            caseno,severity,queue,casetime = event.split('-~')
            #print("Comparing {} {} {} {} to {} ".format(year,index,event,casetime,date))
            if parser.parse(casetime).date() < parser.parse(date).date():
                #print("Found Insert Inde-1 {}".format(index))
                insert_index = index
                break
        insert_index = len(AddedDateInQueueEvents[year])-insert_index
        #print("Found Insert Index-2 {}".format(insert_index))
        for event in AddList[::-1]:
            AddedDateInQueueEvents[year].insert(insert_index,event)
    return AddedDateInQueueEvents

date = (datetime.datetime.today() - datetime.timedelta(days=1)).strftime("%Y-%m-%d")
AddedDateInQueueEvents = AddInQueueEventByDateFromFile(date,AddEventList,RemovedInQueueEvents)
AddedDateInQueueEvents[2020][-65:]

initialize insert_index 5


['689238498-~3-~EME-ACI-Solutions-~2020-06-08 06:01:13',
 '689239839-~3-~EME-ACI-Solutions-~2020-06-08 06:02:42',
 '689154452-~1-~FTS-~2020-06-08 06:05:10',
 '689239934-~3-~EME-ACI-Solutions-~2020-06-08 06:25:57',
 '689185699-~3-~EME-ACI-Solutions-~2020-06-08 06:42:10',
 '689240082-~1-~EME-ACI-Solutions-~2020-06-08 06:55:41',
 '689240082-~1-~EME-ACI-Solutions-~2020-06-08 07:24:56',
 '689224906-~3-~FTS-~2020-06-08 07:28:48',
 '689240783-~3-~EME-ACI-Solutions-~2020-06-08 08:45:19',
 '689183304-~3-~UC-~2020-06-08 09:11:04',
 '689241006-~3-~EME-ACI-Solutions-~2020-06-08 09:13:54',
 '689241320-~3-~EME-ACI-Solutions-~2020-06-08 10:05:34',
 '689241375-~3-~EME-ACI-Solutions-~2020-06-08 10:15:01',
 '689241675-~3-~EME-ACI-Solutions-~2020-06-08 11:07:06',
 '689182716-~2-~EME-ACI-Solutions-~2020-06-08 11:11:28',
 '689233408-~2-~EME-ACI-Solutions-~2020-06-08 11:59:13',
 '689154452-~1-~FTS-~2020-06-09 06:07:20',
 '689212170-~3-~EME-ACI-Solutions-~2020-06-09 06:07:22',
 '689154452-~1-~UC-~2020-06-09 

In [293]:
import glob
###### Accept file parsing Read the current Parsing file 2020_InQueue.txt ######
jupyter_container_path = '/home/jovyan'
flask_container_path = ''
current_container_path = jupyter_container_path
eventfilenames = current_container_path + '/HTTSDashboard/logs/ACI/events/*_AcceptEvent.txt'
eventfilenames = current_container_path + '/HTTSDashboard/logs/TAC/EMEA/ACI/events/*_AcceptEvent.txt'
acceptfilelist = glob.glob(eventfilenames)

def FindAcceptEventsFromEventFile(acceptfilelist):
    if len(acceptfilelist) == 0:
        return
    AcceptEvents_by_year = {}
    ###### Accept file parsing Read the current Parsing file 2020_InQueue.txt ######
    for acceptfile in acceptfilelist:
        print(acceptfile)
        with open(acceptfile,'r') as f:
            ReadAcceptEvents = f.readlines()
        f.close()
        ReadAcceptEvents = [event.strip() for event in ReadAcceptEvents if event.strip()]
        for event in ReadAcceptEvents:
            caseno, casesev, ccoid,workgroup,timestamp = event.split("-~")
            caseyear = parser.parse(timestamp).year
            if caseyear not in AcceptEvents_by_year.keys():
                AcceptEvents_by_year[caseyear] = []
            AcceptEvents_by_year[caseyear].append(event)
    return AcceptEvents_by_year
    
AcceptEvents_by_year = FindAcceptEventsFromEventFile(acceptfilelist)
AcceptEvents_by_year[2020][-20:]

/home/jovyan/HTTSDashboard/logs/TAC/EMEA/ACI/events/2020_AcceptEvent.txt


['689183304-~3-~kattodor-~UC_WORKGROUP-~2020-06-08 09:11:28',
 '689240989-~3-~amatahen-~EMEAR-ORCH-ACI-~2020-06-08 09:12:42',
 '689241006-~3-~kdoodi-~GCE-ACI-Solutions-~2020-06-08 09:16:21',
 '689241321-~3-~lowtouch-~VIRTUAL-~2020-06-08 10:05:34',
 '689241183-~2-~calonzom-~EMEAR-ORCH-ACI-~2020-06-08 10:19:28',
 '689241320-~3-~visgupt2-~GCE-ACI-Solutions-~2020-06-08 10:20:36',
 '689241375-~3-~prpratee-~APT-ACI-SOLUTIONS2-~2020-06-08 10:21:41',
 '689241373-~3-~dashmari-~HTTS-EMEAR-DC-ACI-~2020-06-08 10:24:12',
 '689241520-~4-~halsaudi-~AMM-ACI-~2020-06-08 10:45:18',
 '689241469-~1-~dgomezbe-~UC_WORKGROUP-~2020-06-08 10:45:45',
 '689241624-~3-~amatahen-~EMEAR-ORCH-ACI-~2020-06-08 10:57:26',
 '689233373-~3-~aafaneh-~AMM-ACI-~2020-06-08 11:11:30',
 '689182716-~2-~deseth2-~GCE-ACI-Solutions-~2020-06-08 11:12:21',
 '689241686-~3-~tadepoju-~EMEAR-ORCH-ACI-~2020-06-08 11:16:41',
 '689241673-~3-~zaldawou-~EMEAR-ORCH-ACI-~2020-06-08 11:23:24',
 '689182716-~2-~mripatel-~UC_WORKGROUP-~2020-06-08 11

In [123]:
import copy

date = datetime.datetime.today().strftime("%Y-%m-%d")
######Function to remove a speicifc date accept events ######
def RemoveAcceptEventByDateFromFile(date,AcceptEvents):
    if not AcceptEvents:
        return {}
    ######Need to deep copy the list ######
    RemovedDateAcceptEvents = copy.deepcopy(AcceptEvents)
    for year,events in AcceptEvents.items():
        for index,event in enumerate(AcceptEvents[year][::-1]):
            caseno,severity,ccoid,workgroup,casetime = event.split('-~')
            if parser.parse(casetime).strftime("%Y-%m-%d") == date:
                #print("Removing AcceptEvents {} - {} ".format(date,event.strip()))
                RemovedDateAcceptEvents[year].pop(len(AcceptEvents[year])-index-1)
            if parser.parse(casetime) < parser.parse(date):
                break
    return RemovedDateAcceptEvents

date = (datetime.datetime.today() - datetime.timedelta(days=1)).strftime("%Y-%m-%d")
RemovedDateAcceptEvents = RemoveAcceptEventByDateFromFile(date,AcceptEvents_by_year)
RemovedDateAcceptEvents[2020][-40:]

['689126972-~3-~tskanai-~JAPAN-ACI-~2020-05-21 00:16:22',
 '689114061-~3-~gauvasud-~UC_WORKGROUP-~2020-05-21 00:25:09',
 '688534492-~3-~maveer-~UC_WORKGROUP-~2020-05-21 00:45:29',
 '689124387-~2-~siddhp-~APT-ACI-SOLUTIONS-~2020-05-21 01:11:43',
 '689133456-~2-~tonzeng-~APT-ACI-SOLUTIONS-~2020-05-21 01:15:57',
 '689130460-~2-~reperuma-~APT-ACI-SOLUTIONS2-~2020-05-21 01:25:28',
 '689133555-~3-~piguo-~APJ-CHINA-GCC-~2020-05-21 01:41:56',
 '689133894-~3-~dayamamo-~JAPAN-NEXUS-~2020-05-21 02:15:49',
 '689133924-~3-~lowtouch-~VIRTUAL-~2020-05-21 02:19:53',
 '689133555-~3-~dawang-~APT-GC-Solution-~2020-05-21 02:21:42',
 '689133894-~3-~ykawano-~JAPAN-HTTS-FLETS-~2020-05-21 02:35:19',
 '689133999-~2-~kahande-~APT-ACI-SOLUTIONS2-~2020-05-21 02:45:58',
 '689133923-~3-~pli3-~APT-GC-Solution-~2020-05-21 02:47:05',
 '689134015-~3-~lchuan-~APT-GC-Solution-~2020-05-21 02:58:00',
 '689134064-~3-~lowtouch-~VIRTUAL-~2020-05-21 02:57:20',
 '689134052-~3-~raghb-~APT-ACI-SOLUTIONS2-~2020-05-21 02:57:30',
 '

In [120]:
AddEventList = AcceptEventInOrderList

######Function to add a speicifc date inqueue inorder events ######
def AddAcceptEventByDateFromFile(date, AddList, AcceptEvents):
    if not AcceptEvents:
        AcceptEvents = {}
    if parser.parse(date).year not in AcceptEvents.keys():
        AcceptEvents[parser.parse(date).year] = []
    ######Need to deep copy the list ######
    AddedDateAcceptEvents = copy.deepcopy(AcceptEvents)
    insert_index = len(AcceptEvents[parser.parse(date).year])
    
    for year, events in AcceptEvents.items():
        for index,event in enumerate(events[::-1]):
            caseno,severity,ccoid,workgroup, casetime = event.split('-~')
            #print(event.strip())
            if parser.parse(casetime).date() < parser.parse(date).date():
                #print("Insert Index Found {}-{} {}".format(parser.parse(casetime).date(),parser.parse(date).date(),insert_index))
                insert_index = index
                break
        insert_index = len(AddedDateAcceptEvents[year])-insert_index
        #print("Insert Index {}".format(insert_index))
        #print(AcceptEvents[year][insert_index])
        
        for event in AddList[::-1]:
            #print("Adding AcceptEvent {} - {} at {}".format(date,event,insert_index))
            AddedDateAcceptEvents[year].insert(insert_index,event)
    return AddedDateAcceptEvents

date = (datetime.datetime.today() + datetime.timedelta(days=-1)).strftime("%Y-%m-%d")
AddedDateAcceptEvents = AddAcceptEventByDateFromFile(date,AddEventList,RemovedDateAcceptEvents)
AddedDateAcceptEvents[2020][-40:]

689148383-~2-~kahande-~APT-ACI-SOLUTIONS2-~2020-05-23 01:06:34
689146430-~3-~siddhp-~APT-ACI-SOLUTIONS-~2020-05-23 00:36:17
689136652-~3-~minkwong-~APT-ACI-SOLUTIONS-~2020-05-23 00:16:57
689111638-~3-~roagraw2-~UC_WORKGROUP-~2020-05-23 00:09:50
687954518-~3-~lindawa-~UC_WORKGROUP-~2020-05-21 05:52:16
Insert Index Found 2020-05-21-2020-05-22 1
Insert Index 2294
689111638-~3-~roagraw2-~UC_WORKGROUP-~2020-05-23 00:09:50

Adding AcceptEvent 2020-05-22 - 689141508-~3-~ablazrod-~US-ACI-SOL-WEST-~2020-05-22 00:06:33 at 4
Adding AcceptEvent 2020-05-22 - 689135494-~3-~mkamijo-~JAPAN-ACI-~2020-05-22 00:12:19 at 4
Adding AcceptEvent 2020-05-22 - 689135312-~3-~yushimaz-~JAPAN-ACI-~2020-05-22 00:18:30 at 4
Adding AcceptEvent 2020-05-22 - 689133999-~2-~kahande-~APT-ACI-SOLUTIONS2-~2020-05-22 00:26:44 at 4
Adding AcceptEvent 2020-05-22 - 689128378-~3-~vanakkad-~UC_WORKGROUP-~2020-05-22 00:26:14 at 4
Adding AcceptEvent 2020-05-22 - 689140726-~1-~wilchong-~UC_WORKGROUP-~2020-05-22 00:37:30 at 4
Adding 

['689134406-~3-~deepaky-~APT-ACI-SOLUTIONS2-~2020-05-21 04:56:14\n',
 '689134528-~2-~junwa-~APT-ACI-SOLUTIONS-~2020-05-21 05:44:37\n',
 '687954518-~3-~lindawa-~UC_WORKGROUP-~2020-05-21 05:52:16\n',
 '689141508-~3-~ablazrod-~US-ACI-SOL-WEST-~2020-05-22 00:06:33',
 '689135494-~3-~mkamijo-~JAPAN-ACI-~2020-05-22 00:12:19',
 '689135312-~3-~yushimaz-~JAPAN-ACI-~2020-05-22 00:18:30',
 '689133999-~2-~kahande-~APT-ACI-SOLUTIONS2-~2020-05-22 00:26:44',
 '689128378-~3-~vanakkad-~UC_WORKGROUP-~2020-05-22 00:26:14',
 '689140726-~1-~wilchong-~UC_WORKGROUP-~2020-05-22 00:37:30',
 '689110886-~3-~tonzeng-~APT-ACI-SOLUTIONS-~2020-05-22 00:40:25',
 '689137507-~3-~prpratee-~APT-ACI-SOLUTIONS2-~2020-05-22 00:41:55',
 '689124387-~2-~zmeng-~CX-APJC-SYD-DC-HTEC-~2020-05-22 00:41:46',
 '689139346-~3-~junwa-~APT-ACI-SOLUTIONS-~2020-05-22 00:46:17',
 '689140386-~3-~reperuma-~APT-ACI-SOLUTIONS2-~2020-05-22 00:45:46',
 '689128378-~3-~gauvasud-~UC_WORKGROUP-~2020-05-22 00:44:46',
 '689139008-~1-~deepaky-~UC_WORKGRO